In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}

In [3]:

main_webUrl = "https://www.rumah123.com/jual/sumatera-utara/rumah/?page=1"

req_main = requests.get(main_webUrl, headers=headers)

In [4]:
main_html = BeautifulSoup(req_main.content, "html5lib")

In [5]:
max_page = main_html.find_all("li", attrs={"class": "ui-molecule-paginate__item"})
max_page = int(max_page[-1].text)
max_page

421

In [6]:
dataframe = {}
dataframe["Nama"] = []
dataframe["Harga"] = []

In [7]:
for i in range(1, 200):
    webUrl = f"https://www.rumah123.com/jual/sumatera-utara/rumah/?page={i}"
    req = requests.get(webUrl, headers=headers)
    
    html = BeautifulSoup(req.content, "html5lib")
    time.sleep(2)
    
    link_sections = html.find_all("div", attrs={"class" : "intersection-card-container"})

    for link_sec in link_sections:
        link = link_sec.find("h2").parent["href"]
        
        if "perumahan-baru" in link:
            continue
        try:
            sub_url = requests.get("https://www.rumah123.com" + link, headers=headers)
            sub_html = BeautifulSoup(sub_url.content, "html5lib")
            time.sleep(2)

            title = sub_html.find("h1", attrs={"class" :"text-gray-800 text mb-5"})
            dataframe["Nama"].append(title.text)
            
            specific_div = sub_html.find("div", attrs={"class" : "w-full md:w-[750px] self-stretch"})
            harga = specific_div.find("span")
            dataframe["Harga"].append(harga.text)

            property_div = sub_html.find("div",attrs={"id" : "property-information"})

            propertyes = [[prob.text,prob.nextSibling.text] for prob in property_div.find_all("p", attrs={"class":"w-32"})]
        except:
            continue
        
        print(link)
        for prop in propertyes:
            prop_key = prop[0]
            prop_value = prop[1]
            
            if not prop_key in dataframe.keys():
                dataframe[prop_key] = []
            
            if prop_key in dataframe.keys():
                dataframe[prop_key].append(prop_value)
            
        print("")


/properti/medan/hos12231232/

/properti/medan/hos18935826/

/properti/medan/hos19028452/

/properti/medan/hos19177570/

/properti/deli-serdang/hos19179261/

/properti/medan/hos19394014/

/properti/medan/hos18346899/

/properti/medan/hos18215108/

/properti/medan/hos18346896/

/properti/pematang-siantar/hos18215119/

/properti/medan/hos19245888/

/properti/medan/hos18697317/

/properti/medan/hos16448817/

/properti/medan/hos16064771/

/properti/medan/hos19323924/

/properti/medan/hos19361776/

/properti/deli-serdang/hos18116792/

/properti/medan/hos16064811/

/properti/medan/hos16064827/

/properti/medan/hos19357271/

/properti/medan/hos19259880/

/properti/deli-serdang/hos17795879/

/properti/medan/hos18140304/

/properti/medan/hos18963892/

/properti/medan/hos17563119/

/properti/medan/hos18169040/

/properti/medan/hos17814911/

/properti/medan/hos17779149/

/properti/medan/hos19383993/

/properti/medan/hos19369798/

/properti/medan/hos19062207/

/properti/medan/hos17814911/

/propert

In [8]:
rumah123_df = pd.DataFrame.from_dict(dataframe, orient='index').T
rumah123_df = rumah123_df.fillna(0)
rumah123_df.to_json("Provinsi_SumatraUtara.json", orient="records")

In [9]:
rumah123_df

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Jumlah Lantai,Kondisi Properti,Garasi
0,"Rumah Bagus Dipusat Kota, Harga Miring, Banyak...","Rp 3,8 Miliar",5,5,350 m²,300 m²,2,SHM,Lainnya,1,1,2,Bagus,2
1,Dijual Rumah Mewah Sangat Luas Dengan Konsep M...,"Rp 1,75 Miliar",4,3,490 m²,202 m²,1,SHM,2200 Watt,1,1,1,Bagus,1
2,Dijual Rumah Luas 2 Lantai Siap Huni di Medan ...,Rp 590 Juta,6,2,176 m²,264 m²,1,SHM,900 Watt,1,1,2,Bagus,2
3,Dijual Rumah Strategis dekat Gaperta Ujung Hel...,"Rp 1,47 Miliar",3,2,278 m²,112 m²,1,SHM,900 Watt,1,1,1,Bagus,1
4,Rumah Komp.Merci Barn Cluster Roterdam Type Av...,Rp 650 Juta,3,2,75 m²,76 m²,1,SHM,Lainnya,1,1,2,Bagus,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1858,Rumah Inti Kota Jl. Tilak. Strategis. Dekat Th...,"Rp 1,65 Miliar",0,0,72 m²,132 m²,0,SHM,0,0,0,0,0,0
1859,Dijual Rumah Tanah Gudang Hm Joni Medan,Rp 42 Miliar,0,0,320 m²,208 m²,0,SHM,0,0,0,0,0,0
1860,"Rumah 3 Lantai di Jalan Budi Utomo, Medan","Rp 1,18 Miliar",0,0,260 m²,525 m²,0,SHM,0,0,0,0,0,0
1861,Dijual Rumah Siap Huni Daerah Deli Serdang,"Rp 2,6 Miliar",0,0,0,0,0,SHM,0,0,0,0,0,0
